<a href="https://colab.research.google.com/github/shv-om/Cryptology/blob/main/DES_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversion to Binary

In [1]:
def conbin(msg):
  newstream = []
  for m in msg:
    b = format(int(m, 16), '04b')
    newstream.append(b.replace('0b', ''))
  
  return ''.join(newstream)

In [2]:
msgstream = list(conbin('0123456789abcdef'))
print(msgstream)
print(len(msgstream))

['0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1']
64


# Permutation

In [33]:
def calc_permutation(stream, pmatrix):
  pmshape = pmatrix.shape
  newstream = np.empty(pmshape, str)
  
  for i in range(pmshape[0]):
    for j in range(pmshape[1]):
      # print(i, j, '-->', pmatrix[i][j]-1)
      newstream[i][j] = stream[pmatrix[i][j]-1]

  return newstream

In [42]:
# Splitting stream into Left and Right stream
def split_left_right(stream):
  left_part = stream[:len(stream)//2]
  right_part = stream[len(stream)//2:]

  return left_part, right_part

# Initial Permutation

In [29]:
import numpy as np

IP = np.array([[58, 50, 42, 34, 26, 18, 10, 2],
              [60, 52, 44, 32, 28, 20, 12, 4],
              [62, 54, 46, 38, 30, 22, 14, 6],
              [64, 56, 48, 40, 32, 24, 16, 8],
              [57, 49, 41, 33, 25, 17, 9, 1],
              [59, 51, 43, 35, 27, 19, 11, 3],
              [61, 53, 45, 37, 29, 21, 13, 5],
              [63, 55, 47, 39, 31, 23, 15, 7]])

IP

array([[58, 50, 42, 34, 26, 18, 10,  2],
       [60, 52, 44, 32, 28, 20, 12,  4],
       [62, 54, 46, 38, 30, 22, 14,  6],
       [64, 56, 48, 40, 32, 24, 16,  8],
       [57, 49, 41, 33, 25, 17,  9,  1],
       [59, 51, 43, 35, 27, 19, 11,  3],
       [61, 53, 45, 37, 29, 21, 13,  5],
       [63, 55, 47, 39, 31, 23, 15,  7]])

In [34]:
# Initial Permutation
initstream = calc_permutation(msgstream, IP)
initstream

array([['1', '1', '0', '0', '1', '1', '0', '0'],
       ['0', '0', '0', '1', '0', '0', '0', '0'],
       ['1', '1', '0', '0', '1', '1', '0', '0'],
       ['1', '1', '1', '1', '1', '1', '1', '1'],
       ['1', '1', '1', '1', '0', '0', '0', '0'],
       ['1', '0', '1', '0', '1', '0', '1', '0'],
       ['1', '1', '1', '1', '0', '0', '0', '0'],
       ['1', '0', '1', '0', '1', '0', '1', '0']], dtype='<U1')

In [47]:
MsgLeft, MsgRight = split_left_right(initstream)
MsgLeft, MsgRight

(array([['1', '1', '0', '0', '1', '1', '0', '0'],
        ['0', '0', '0', '1', '0', '0', '0', '0'],
        ['1', '1', '0', '0', '1', '1', '0', '0'],
        ['1', '1', '1', '1', '1', '1', '1', '1']], dtype='<U1'),
 array([['1', '1', '1', '1', '0', '0', '0', '0'],
        ['1', '0', '1', '0', '1', '0', '1', '0'],
        ['1', '1', '1', '1', '0', '0', '0', '0'],
        ['1', '0', '1', '0', '1', '0', '1', '0']], dtype='<U1'))

# Key Generation

In [28]:
keystream = list(conbin('1234abfce5872cb9'))
print(keystream)
print(len(keystream))

['0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1']
64


In [38]:
pc1 = np.array([[57, 49, 41, 33, 25, 17, 9],
                [1, 58, 50, 42, 34, 26, 18],
                [10, 2, 59, 51, 43, 35, 27],
                [19, 11, 3, 60, 52, 44, 36],
                [63, 55, 47, 39, 31, 23, 15],
                [7, 62, 54, 46, 38, 30, 22],
                [14, 6, 61, 53, 45, 37, 29],
                [21, 13, 5, 28, 20, 12, 4]])

pc1

array([[57, 49, 41, 33, 25, 17,  9],
       [ 1, 58, 50, 42, 34, 26, 18],
       [10,  2, 59, 51, 43, 35, 27],
       [19, 11,  3, 60, 52, 44, 36],
       [63, 55, 47, 39, 31, 23, 15],
       [ 7, 62, 54, 46, 38, 30, 22],
       [14,  6, 61, 53, 45, 37, 29],
       [21, 13,  5, 28, 20, 12,  4]])

In [40]:
key_permuted = calc_permutation(keystream, pc1)
key_permuted

array([['1', '0', '1', '1', '1', '1', '0'],
       ['0', '0', '0', '0', '1', '1', '0'],
       ['0', '0', '1', '1', '0', '1', '1'],
       ['1', '1', '0', '1', '0', '0', '0'],
       ['0', '0', '1', '0', '0', '1', '0'],
       ['1', '0', '1', '1', '1', '1', '0'],
       ['1', '0', '1', '1', '0', '0', '1'],
       ['1', '0', '0', '1', '0', '1', '1']], dtype='<U1')

In [49]:
KeyLeft, KeyRight = split_left_right(key_permuted)
KeyLeft, KeyRight

(array([['1', '0', '1', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '1', '1', '0'],
        ['0', '0', '1', '1', '0', '1', '1'],
        ['1', '1', '0', '1', '0', '0', '0']], dtype='<U1'),
 array([['0', '0', '1', '0', '0', '1', '0'],
        ['1', '0', '1', '1', '1', '1', '0'],
        ['1', '0', '1', '1', '0', '0', '1'],
        ['1', '0', '0', '1', '0', '1', '1']], dtype='<U1'))

In [7]:
FP = np.array([[40, 8, 48, 16, 56, 24, 64, 32],
              [39, 7, 47, 15, 55, 23, 63, 31],
              [38, 6, 46, 14, 54, 22, 62, 30],
              [37, 5, 45, 13, 53, 21, 61, 29],
              [36, 4, 44, 12, 52, 20, 60, 28],
              [35, 3, 43, 11, 51, 19, 59, 27],
              [34, 2, 42, 10, 50, 18, 58, 26],
              [33, 1, 41, 9, 49, 17, 57, 25]])